In [ ]:
import os
import sunra_client
import requests
from PIL import Image
import matplotlib.pyplot as plt
from io import BytesIO
import time
import getpass
from IPython.display import Video, HTML

# Set up API key if not already configured
if 'SUNRA_KEY' not in os.environ:
    api_key = getpass.getpass("Enter your Sunra API key: ")
    os.environ['SUNRA_KEY'] = api_key

# Configure the client
sunra_client.config(credentials=os.environ['SUNRA_KEY'])
print("✓ Sunra client configured successfully!")


In [ ]:
def display_image(image_url, title="Generated Image", size=(10, 10)):
    """Download and display an image from a URL"""
    try:
        response = requests.get(image_url)
        img = Image.open(BytesIO(response.content))
        
        plt.figure(figsize=size)
        plt.imshow(img)
        plt.axis('off')
        plt.title(title)
        plt.show()
        
        print(f"Image size: {img.size}")
        print(f"Image format: {img.format}")
        return img
        
    except Exception as e:
        print(f"Error displaying image: {e}")
        print(f"You can view the image directly at: {image_url}")
        return None

def display_video(video_url, title="Generated Video"):
    """Display a video from a URL"""
    try:
        print(f"Video URL: {video_url}")
        print(f"Title: {title}")
        
        # Create HTML video element
        video_html = f"""
        <video width="640" height="480" controls>
            <source src="{video_url}" type="video/mp4">
            Your browser does not support the video tag.
        </video>
        """
        
        display(HTML(video_html))
        
    except Exception as e:
        print(f"Error displaying video: {e}")
        print(f"You can view the video directly at: {video_url}")

def generate_image_to_image(prompt, input_image_url, strength=0.8, show_result=True):
    """Generate an image-to-image transformation"""
    
    print(f"Generating image-to-image with prompt: '{prompt}'")
    print(f"Input image: {input_image_url}")
    print("Please wait...")
    
    try:
        result = sunra_client.subscribe(
            "black-forest-labs/flux-kontext-pro/image-to-image",
            arguments={
                "prompt": prompt,
                "image": input_image_url,
                "strength": strength,
                "prompt_enhancer": False,
                "seed": 0,
                "aspect_ratio": "1:1",
                "output_format": "jpeg",
                "safety_tolerance": 6
            },
            with_logs=True,
            on_enqueue=lambda req_id: print(f"✓ Request enqueued: {req_id}"),
            on_queue_update=lambda status: print(f"Status: {status}"),
        )
        
        if result.get('images'):
            image_url = result['images'][0]['url']
            print(f"✓ Image transformation completed!")
            print(f"Result URL: {image_url}")
            
            if show_result:
                display_image(image_url, f"Transformed: {prompt}")
            
            return result
        else:
            print("No image generated")
            return None
            
    except Exception as e:
        print(f"❌ Error generating image: {e}")
        return None

def generate_image_to_video(prompt, input_image_url, duration=5, guidance_scale=0.5, show_result=True):
    """Generate a video from an image"""
    
    print(f"Generating video with prompt: '{prompt}'")
    print(f"Input image: {input_image_url}")
    print(f"Duration: {duration} seconds")
    print("Please wait... (this may take longer than image generation)")
    
    try:
        result = sunra_client.subscribe(
            "kling/kling-v2-master/image-to-video",
            arguments={
                "prompt": prompt,
                "negative_prompt": "",
                "guidance_scale": guidance_scale,
                "aspect_ratio": "16:9",
                "duration": duration,
                "start_image": input_image_url
            },
            with_logs=True,
            on_enqueue=lambda req_id: print(f"✓ Request enqueued: {req_id}"),
            on_queue_update=lambda status: print(f"Status: {status}"),
        )
        
        if result.get('videos'):
            video_url = result['videos'][0]['url']
            print(f"✓ Video generation completed!")
            print(f"Video URL: {video_url}")
            
            if show_result:
                display_video(video_url, f"Video: {prompt}")
            
            return result
        else:
            print("No video generated")
            return None
            
    except Exception as e:
        print(f"❌ Error generating video: {e}")
        return None

print("Helper functions ready!")


In [ ]:
# Generate a base image to work with
base_prompt = "a simple portrait of a person in a garden"

try:
    result = sunra_client.subscribe(
        "black-forest-labs/flux-kontext-pro/text-to-image",
        arguments={
            "prompt": base_prompt,
            "prompt_enhancer": False,
            "seed": 42,
            "aspect_ratio": "1:1",
            "output_format": "jpeg",
            "safety_tolerance": 6
        },
        with_logs=True,
        on_enqueue=lambda req_id: print(f"✓ Request enqueued: {req_id}"),
        on_queue_update=lambda status: print(f"Status: {status}"),
    )
    
    if result.get('images'):
        base_image_url = result['images'][0]['url']
        print(f"✓ Base image generated!")
        print(f"Base image URL: {base_image_url}")
        
        display_image(base_image_url, "Base Image: " + base_prompt)
    else:
        print("No base image generated")
        base_image_url = None
        
except Exception as e:
    print(f"❌ Error generating base image: {e}")
    base_image_url = None


In [ ]:
# Use our generated image or a sample image
if base_image_url:
    input_image = base_image_url
else:
    # Use a sample image from the original demo
    input_image = "https://assets.sunra.ai/uploads/1748811753168-05ceab0d.png"
    print("Using sample image for transformation")
    display_image(input_image, "Sample Input Image")

# Transform the image with different prompts
transformations = [
    "convert to a watercolor painting style",
    "make it look like a renaissance painting",
    "transform into a cyberpunk style with neon colors",
    "convert to a pencil sketch drawing"
]

for i, transform_prompt in enumerate(transformations, 1):
    print(f"\n=== Transformation {i}: {transform_prompt} ===")
    result = generate_image_to_image(transform_prompt, input_image, strength=0.8)


In [ ]:
# Generate a video from our image
video_prompt = "A cute hamster lies leisurely on a lifebuoy, wearing fashionable sunglasses, and drifts with the gentle waves on the shimmering sea surface. The hamster reclines comfortably, enjoying a peaceful and pleasant time. Cartoon style, the camera follows the subject moving, with a heartwarming and high picture quality."

# Use the sample image from the original demo for consistency
sample_image = "https://assets.sunra.ai/uploads/1748811753168-05ceab0d.png"

print("Generating video from image...")
print("This may take several minutes, please be patient...")

video_result = generate_image_to_video(
    video_prompt, 
    sample_image, 
    duration=5, 
    guidance_scale=0.5
)


In [ ]:
# Compare different strength values
test_image = "https://assets.sunra.ai/uploads/1748811753168-05ceab0d.png"
test_prompt = "transform into a magical fairy tale illustration"

strength_values = [0.3, 0.5, 0.8, 1.0]

print("Comparing different strength values:")
print("Original image:")
display_image(test_image, "Original Image")

for strength in strength_values:
    print(f"\n=== Strength: {strength} ===")
    result = generate_image_to_image(test_prompt, test_image, strength=strength)


In [ ]:
# Different video generation approaches
video_test_image = "https://assets.sunra.ai/uploads/1748811753168-05ceab0d.png"

video_experiments = [
    {
        "prompt": "gentle waves moving in slow motion",
        "duration": 3,
        "guidance_scale": 0.3
    },
    {
        "prompt": "floating peacefully with subtle camera movement",
        "duration": 5,
        "guidance_scale": 0.5
    },
    {
        "prompt": "dramatic ocean scene with dynamic camera angles",
        "duration": 5,
        "guidance_scale": 0.8
    }
]

print("Generating videos with different parameters...")
print("Note: Each video may take several minutes to generate")

for i, experiment in enumerate(video_experiments, 1):
    print(f"\n=== Video Experiment {i} ===")
    print(f"Prompt: {experiment['prompt']}")
    print(f"Duration: {experiment['duration']}s")
    print(f"Guidance Scale: {experiment['guidance_scale']}")
    
    # Note: In practice, you might want to run only one at a time due to processing time
    # Uncomment the line below to generate the video
    # result = generate_image_to_video(experiment['prompt'], video_test_image, 
    #                                 duration=experiment['duration'], 
    #                                 guidance_scale=experiment['guidance_scale'])


In [ ]:
def test_image_url(image_url):
    """Test if an image URL is accessible and valid"""
    try:
        print(f"Testing image URL: {image_url}")
        response = requests.get(image_url)
        
        if response.status_code == 200:
            img = Image.open(BytesIO(response.content))
            print(f"✓ Image accessible!")
            print(f"  Format: {img.format}")
            print(f"  Size: {img.size}")
            print(f"  Mode: {img.mode}")
            
            # Display the image
            display_image(image_url, "Your Image")
            return True
        else:
            print(f"❌ HTTP Error: {response.status_code}")
            return False
            
    except Exception as e:
        print(f"❌ Error: {e}")
        return False

# Test with a sample image
sample_url = "https://assets.sunra.ai/uploads/1748811753168-05ceab0d.png"
test_image_url(sample_url)

# To test your own image, uncomment and modify the line below:
# test_image_url("YOUR_IMAGE_URL_HERE")
